In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nba_api.stats.endpoints import playbyplayv2, boxscoresummaryv2, playergamelogs, shotchartdetail
from time import sleep

In [8]:
# eh
# gets the shots of player's teammates in a game's quarter
# @param int player_id id of player in nba_api 
# @param bool home_team if the player's team is the home team in a game
# @param list teammates_ids list of teammates of interests' ids
# @param dataframe pbp_data play-by-play data of a quarter
# @param bool starting if the player is starting
# @return dict teammate id to 2pm, 2pa, 3pm, 3pa during player's time on court
def get_shots_in_quarter(player_id, home_team, teammates_ids, pbp_data, starting):
    opd = {}
    for e in teammates_ids:
        opd[e] = [0,0,0,0]
    
    p_in = starting
    which_desc = "HOMEDESCRIPTION" if home_team else "VISITORDESCRIPTION"
    for _, row in pbp_data.iterrows():
        desc = row[which_desc]
        clock = row["PCTIMESTRING"]
        event_type = row["EVENTMSGTYPE"]
        p1 = row["PLAYER1_ID"]
        if (desc!= None):
            if event_type == 8:
                sub_out = row["PLAYER1_ID"]
                sub_in = row["PLAYER2_ID"]
                if sub_in == player_id:
                    p_in = True
                if sub_out == player_id:
                    p_in = False
            if (p_in):
                #made shot
                if ((event_type == 1) and (p1 in teammates_ids)):
                    if " 3PT " in desc:
                        opd[p1][3] = opd[p1][3] + 1 
                        opd[p1][2] = opd[p1][2] + 1 
                    else:
                        opd[p1][0] = opd[p1][0] + 1 
                        opd[p1][1] = opd[p1][1] + 1 
                #missed shot
                if ((event_type == 2) and (p1 in teammates_ids)):
                    if " 3PT " in desc:
                        opd[p1][3] = opd[p1][3] + 1 
                    else:
                        opd[p1][1] = opd[p1][1] + 1 
                
    return(opd)

In [9]:
# eh
# checks if player started a quarter
# @param int player_id id of player in nba_api 
# @param dataframe pbp_data play-by-play data of a quarter
# @param bool home_team if the player's team is the home team in a game
# @return bool if the player started that quarter
def check_start(player_id, pbp_data, home_team):
    which_desc = "HOMEDESCRIPTION" if home_team else "VISITORDESCRIPTION"
    filtered_data = pbp_data[pbp_data[which_desc].notna()]
    for _,row in filtered_data.iterrows():
        if row["EVENTMSGTYPE"] == 8:
            if player_id == row["PLAYER1_ID"]:
                return True
            elif player_id == row["PLAYER2_ID"]:
                return False
        if (row["PLAYER1_ID"] == player_id) or (row["PLAYER2_ID"] == player_id) or (row["PLAYER3_ID"] == player_id):
            return True
    return False

In [10]:
# eh
# gets all shots of interest from a game
# @param int player_id id of player in nba_api 
# @param bool home_team if the player's team is the home team in a game
# @param list teammates_ids list of teammates of interests' ids
# @param int game_id the id of the game of interest
# @return dict all shots of interest in the game
def get_team_shots_game(player_id, home_team, teammates_ids, game_id):
    master_shots = {}
    for e in teammates_ids:
        master_shots[e] = [0,0,0,0]
    for period in range(1,6):
        pbp_data = playbyplayv2.PlayByPlayV2(game_id=game_id, 
                                        start_period = period, 
                                        end_period = period).get_data_frames()[0]
        period_dict = get_shots_in_quarter(player_id, home_team, teammates_ids , pbp_data, check_start(player_id, pbp_data, home_team))
        for e in teammates_ids:
            master_shots[e] = [sum(x) for x in zip(master_shots[e], period_dict[e])]
    return master_shots

In [11]:
# eh
# gets game_ids of all games the player plays in during a season
# @param int player_id id of player in nba_api 
# @param String season the season of interest
# @return list with game_ids
def get_player_game_ids(player_id, season):
    print("gotten ids")
    game_logs = playergamelogs.PlayerGameLogs(player_id_nullable=player_id, season_nullable=season)
    games_data = game_logs.get_data_frames()[0]
    game_ids = games_data['GAME_ID'].tolist()
    return game_ids

In [12]:
# eh
# gets all shots of interest from a season
# @param int player_id id of player in nba_api 
# @param int team_id id of the team
# @param list teammates_ids list of teammates of interests' ids
# @param String season the season of interest
# @return dict all shots of interest in the season
def get_team_shots_season(player_id, team_id, teammates_ids, season):
    season_shots_d = {}
    for e in teammates_ids:
        season_shots_d[e] = [0,0,0,0]
    game_num = 0
    print("started")
    for game_id in get_player_game_ids(player_id, season):
        game_num += 1
        print("started game " + str(game_num))
        home_team = (team_id == boxscoresummaryv2.BoxScoreSummaryV2(game_id=game_id).game_summary.get_dict()['data'][0][6])
        gd = get_team_shots_game(player_id, home_team, teammates_ids, game_id)
        for e in teammates_ids:
            season_shots_d[e] = [sum(x) for x in zip(season_shots_d[e], gd[e])]
        print("processed game " + str(game_num))
        sleep(5)
    return season_shots_d

In [13]:
player_id = 202331
team_id = 1610612746
teammates_ids = [202331, 201935, 202695, 1626181, 1629611, 1627826, 201566, 1629599]
season = "2023-24"
shots_dict_season = get_team_shots_season(player_id, team_id, teammates_ids, season)

started
gotten ids
started game1
processed game 1
started game2
processed game 2
started game3
processed game 3
started game4
processed game 4
started game5
processed game 5
started game6
processed game 6
started game7
processed game 7
started game8
processed game 8
started game9
processed game 9
started game10
processed game 10
started game11
processed game 11
started game12
processed game 12
started game13
processed game 13
started game14
processed game 14
started game15
processed game 15
started game16
processed game 16
started game17
processed game 17
started game18
processed game 18
started game19
processed game 19
started game20
processed game 20
started game21
processed game 21
started game22
processed game 22
started game23
processed game 23
started game24
processed game 24
started game25
processed game 25
started game26
processed game 26
started game27
processed game 27
started game28
processed game 28
started game29
processed game 29
started game30
processed game 30
started g

{202331: [339, 648, 243, 588],
 201935: [113, 228, 127, 324],
 202695: [319, 546, 100, 235],
 1626181: [98, 172, 89, 169],
 1629611: [122, 202, 55, 145],
 1627826: [277, 437, 0, 0],
 201566: [138, 267, 25, 81],
 1629599: [36, 57, 19, 57]}

{202331: [0.523, 0.413],
 201935: [0.497, 0.381],
 202695: [0.569, 0.417],
 1626181: [0.536, 0.435],
 1629611: [0.633, 0.348],
 1627826: [0.649, 0.0],
 201566: [0.509, 0.273],
 1629599: [0.571, 0.38]}